In [ ]:
df = spark.read.format("delta").load("/mnt/datamount/delta_table")

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
with open('tokenfile', 'w') as f:
    f.write(dbutils.secrets.get(scope="creds", key="dbtoken"))
!databricks configure --host https://adb-7218252456053873.13.azuredatabricks.net/ --token-file tokenfile

In [ ]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the data
data = df.toPandas()

# Define feature columns and target
features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
target = "Survived"

# Split the data into training and testing sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow.set_experiment("/Users/bhaveshkak26122000@gmail.com/my_experiment")

# Start an MLflow run
with mlflow.start_run():
    # Train a machine learning model (Random Forest in this example)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate and log accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model
    mlflow.sklearn.log_model(model, "model")

    # Save the feature columns for reference
    mlflow.log_param("features", features)

    # Register the model in MLflow
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
    registered_model_name = "titanic_model"  # Replace with your desired model name

    # You can specify model tags and description if needed
    model_tags = {"key1": "value1", "key2": "value2"}
    model_description = "Description of the registered model"

    # Register the model with MLflow
    registered_model = mlflow.register_model(model_uri, registered_model_name, tags=model_tags)

    # Print the registered model information
    print(f"Registered model: {registered_model.name} (Version {registered_model.version})")

# You can view the registered model in the MLflow UI.


# You can view the MLflow run in the Databricks UI.


2023/10/23 21:32:49 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.7.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
Successfully registered model 'titanic_model'.
2023/10/23 21:32:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: titanic_model, version 1


Registered model: titanic_model (Version 1)


Created version '1' of model 'titanic_model'.
